# Уеб програмиране. Flask

Съдържание:
- Какво е уеб програмиране ?
- Накратко за HTML, CSS и JavaScript
- Какво е Flask ?
- Просто приложение
- Шаблони
- Routing
- Работа със заявки

## Какво е уеб програмиране ?

## Накратко за HTML, CSS и JavaScript

## Какво е Flask ?

## Просто приложение

Първото нещо, което трябва да направим е да заредим Flask модула. Това става с `from flask import Flask`. След това трябва да създадем обект от тип `Flask`. Това става с `app = Flask(__name__)`. Като първи аргумент на конструктора на `Flask` подаваме името на модула, в който се намираме. Това е необходимо, за да Flask знае къде да търси файловете, които са свързани с приложението. 
След това трябва да дефинираме функции, които ще се изпълняват, при поискване на даден път. За целта използваме декоратора `@app.route`. Това е декоратор, който се използва за да се определи къде да се изпълни функцията. Нека дефинираме функция, която да връща текста `Hello world`, обвит като `<p>` таг. Това става с `return "<p>Hello world</p>"`.

In [2]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

За да стартираме нашето уеб приложение, трябва да изпълним следната команда: `python3 -m flask --app simple_app run`. Това ще стартира нашето приложение на адрес `http://127.0.0.1:5000`

In [3]:
!python3 -m flask --app examples/simple_app run

 * Serving Flask app 'simple_app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Dec/2022 16:38:47] "GET / HTTP/1.1" 200 -
^C


## Шаблони

Flask поддържа и т.нар. HTML шаблони - това са HTML страници, които могат да получават данни от нашия Python код. Flask търси тези template-и в специална папка на име `templates`.  

Ще създадем един базов шаблон, който ще съдържа общата част на всяка наша страница:
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>{{ title }}</title>
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">

    {% block styles %}
    {% endblock %}

    {% block scripts %}
    {% endblock %}
</head>
<body>
    <div id="main-content">
        {% block content %}
        {% endblock %}
    </div>
</body>
</html>
```

Забелязваме някои нестандартни за HTML елементи - всичко, което е оградено от `{}` скоби е израз, който ще бъде изчислен от Python кода. Това са `{{ title }}` и `{% block content %}{% endblock %}`. Първият израз ще бъде заменен със стойността на променливата `title`, а вторият израз ще бъде заменен със съдържанието на блока `content`.

Нека разширим нашия базов шаблон, като направим началната ни страница - тя ще е с името `index.html`

```html
{% extends "base.html" %}

{% block content %}
    <h1>This is the home page</h1> 
    <h2>Hi, {{user}}</h2>
{% endblock %}
```

Остава единствено да кажем на Flask да зареди нашия шаблон - това става с помощта на метода `render_template".

In [1]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
def home():
    return render_template('index.html', title='Home', user='Lyubo')

Освен името на шаблона, към `render_template` можем да подадем и стойности, с които да бъдат заместени изразите в шаблона.

In [3]:
!python3 -m flask --app examples/simple_templates run

 * Serving Flask app 'examples/templates_simple'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Dec/2022 09:54:45] "GET / HTTP/1.1" 200 -
^C


## Routing

Дотук работехме само с една страница - как може да добавим допълнителни страници ?

Можем да използваме `@app.route` декоратора, но с друг път, за да можем да създадем други страници. Нека направим примитивна логин страница.

Нека създадем нов template, който да бъде използван за логин страница.

```html
{% extends "base.html" %}

{% block content %}
    <h1>This is the login page</h1>
    <form action="/login_action" method="post">
        <input type="text" name="username" placeholder="Username">
        <input type="password" name="password" placeholder="Password">
        <input type="submit" value="Login">
    </form>
{% endblock %}
```

In [1]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route("/")
def home():
    return render_template('index.html', title='Home', user='Lyubo')

@app.route("/login")
def login():
    return render_template('login.html', title='Login')

In [2]:
!python3 -m flask --app examples/simple_login run

 * Serving Flask app 'examples/simple_login'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [30/Dec/2022 12:31:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2022 12:31:49] "GET /login HTTP/1.1" 200 -
^C


`/page/<var>`, аргументи към функциите

## Работа с ресурси

Почти винаги ще ни се наложи да работим с някакви статични ресурси в нашето уеб приложение - било то CSS файлове, JS файлове или изображения. За да можем да достъпваме тези ресурси, ще трябва да ги копираме в папка на нашето приложение. 
Нека създадем файла `style.css` в папката `static`.

```css
#main-content {
    width: 100%;
    height: 100%;
    background-color: #333333;
    padding: 10px;
    border: 1px solid #ccc;
    border-radius: 5px;
    box-shadow: 0 0 10px #ccc;
}

h1 {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    color: white;
}
```

За да достъпим такъв статичен ресурс през нашия HTML код, можем да използваме `{{ url_for('static', filename='img/logo.png') }}`. Това ще ни върне пътя до файла `logo.png` в папката `img` в папката `static`.

In [ ]:
!python3 -m flask --app examples/fancy_page run

## Работа със заявки

Ще добавим проста логин форма в нашия уеб сайт.

## JSON API

- make_response
- jsonify
- request
- config.from_envvar
- app.response_class
